In [ ]:
import openvds
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.image as mpimg
import numpy as np
import time
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (16, 16)

## Open a VDS connection and get the Layout object for the connection
Specific OpenOptions has to be used to open a connection to different cloud vendors. The AWS open options also relies on the AWS standardized credentials file in $HOME/.aws/credentials

In [ ]:
error = openvds.Error()
openOptions = openvds.AWSOpenOptions("bucket", "folder_in_bucket", "region")
#openOptions = openvds.AzureOpenOptions("connection_string", "container", "blob") #blob is ignored for now
vds = openvds.open(openOptions, error)
layout = openvds.getLayout(vds)

## Layout
The layout can be used to retrieve properties about the VDS.

In [ ]:
print("ChannelCount: {}".format(layout.getChannelCount()))
for i in range(layout.getChannelCount()):
    print("  Channel Name: {}".format(layout.getChannelName(i)))
    print("    Value range: {} - {}".format(layout.getChannelValueRangeMin(i), layout.getChannelValueRangeMax(i)))
    
print("Dimensionality: {}".format(layout.getDimensionality()))
for i in range(layout.getDimensionality()):
    print("  Dimension name: {}".format(layout.getDimensionName(i)))
    print("    Number of samples: {}".format(layout.getDimensionNumSamples(i)))
    print("    Coordinate min max {} - {}".format(layout.getDimensionMin(i), layout.getDimensionMax(i)))

## Metadata accessor
Metadata can be accessed through the layout objects getMetadataXX function. The type is part of the key and no implicit conversion will occur.
- getMetadataInt
- getMetadataIntVector2
- getMetadataIntVector3
- getMetadataIntVector4
- getMetadataFloat
- getMetadataFloatVector2
- getMetadataFloatVector3
- getMetadataFloatVector4
- getMetadataDouble
- getMetadataDoubleVector2
- getMetadataDoubleVector3
- getMetadataDoubleVector4
- getMetadataString
- getMetadataBLOB




In [ ]:
MetaType = openvds.MetadataType
if layout.isMetadataDoubleVector2Available("SurveyCoordinateSystem", "Origin"):
    print("SurveyCoordinateSystem::Origin: {}".format(layout.getMetadataDoubleVector2("SurveyCoordinateSystem", "Origin")))
if layout.isMetadataDoubleVector2Available("SurveyCoordinateSystem", "InlineSpacing"):
    print("SurveyCoordinateSystem::InlineSpacing: {}".format(layout.getMetadataDoubleVector2("SurveyCoordinateSystem", "InlineSpacing")))
if layout.isMetadataDoubleVector2Available("SurveyCoordinateSystem", "CrosslineSpacing"):
    print("SurveyCoordinateSystem::CrosslineSpacing: {}".format(layout.getMetadataDoubleVector2("SurveyCoordinateSystem", "CrosslineSpacing")))
#layout.getMetadata("SEGY", "TextHeader", openvds.core.MetadataType.BLOB)
#layout.getMetadata("SEGY", "BinaryHeader", openvds.core.MetadataType.BLOB)

## The AccessManager api
The AccessManager api can be used to retieve data from the vds

In [ ]:
accessManager = openvds.VolumeDataAccessManager(vds)
axisDescriptors = [layout.getAxisDescriptor(dim) for dim in range(layout.getDimensionality())]
sliceType = 'inline'
sliceIndex = 55
sliceDimension = 2 if sliceType == 'inline' else 1 if sliceType == 'crossline' else 0 if sliceType == 'timeslice' else 0 if sliceType == 'depthslice' else -1

min = tuple(sliceIndex + 0 if dim == sliceDimension else 0 for dim in range(6))
max = tuple(sliceIndex + 1 if dim == sliceDimension else layout.getDimensionNumSamples(dim) for dim in range(6))

req = accessManager.requestVolumeSubset(min, max, format = openvds.VolumeDataChannelDescriptor.Format.Format_R32)
height = max[0] if sliceDimension != 0 else max[1]
width  = max[2] if sliceDimension != 2 else max[1]

data = req.data.reshape(width, height).transpose()
cmap=plt.get_cmap("seismic")
plt.set_cmap(cmap)
plt.imshow(data)
